In [1]:
import numpy as np
from collections import Counter

In [2]:
train_path=r'F:\study\ml\HonyiLee2020\hw1\data\train.csv'
test_path=r'F:\study\ml\HonyiLee2020\hw1\data\test.csv'

In [3]:
train_data_raw = np.loadtxt(train_path, delimiter= ',', dtype = str, skiprows = 1,encoding='big5')

In [4]:
train_data_raw.shape

(4320, 27)

In [5]:
n_parameters = 18
n_date = len(Counter(train_data_raw[:, 0]).keys())
train_data_raw[train_data_raw == 'NR'] = 0
train_data_raw = train_data_raw[:, 3:].astype(np.float64)
train_data = train_data_raw[:n_parameters, :]
for i in range(1, n_date):
    temp = train_data_raw[(n_parameters * (i - 1)): (n_parameters * i), :]
    train_data = np.hstack((train_data, temp))
n_series = 9
target_row = 9    

In [6]:
mean_x = np.mean(train_data, axis = 1)
std_x = np.std(train_data, axis = 1)

In [7]:
for i in range(n_parameters):
    if std_x[i] != 0:
        train_data[i, :] = (train_data[i, :] - mean_x[i])/std_x[i]

In [8]:
final_data = []
for j in range(train_data.shape[1] - n_series):
    key = train_data[:, j:j+n_series].reshape(18*9, )
    value = train_data[target_row, j+n_series]
    final_data.append([key, value])

In [9]:
train_data=final_data

In [10]:
train_data[0][0].shape

(162,)

In [11]:
w_shape=(1,18*9)

In [12]:
len(train_data)

5751

In [13]:
np.random.shuffle(train_data)
test_data=train_data[:500]
train_data = train_data[500:]

In [14]:
len(train_data)

5251

In [15]:
w = np.zeros(w_shape, dtype = float)
b = 0
eta = 0.0006 #learning rate

In [16]:
n_iter = 1000 #iteration time
train_loss = 0
test_loss = 0
result = []

In [17]:
len(train_data)

5251

In [18]:
for i in range(n_iter):
    train_loss = 0
    for j in range(len(train_data)):
        x = train_data[j][0]
        y = np.dot(w, x) + b
        target = train_data[j][1]

        w_gradient = 2 * (y - target) * x.T
        b_gradient = 2 * (y - target)

        w = w - eta * w_gradient
        b = b - eta * b_gradient

        train_loss += (target - y) ** 2

    result.append(train_loss)

In [19]:
x_train = []
y_train = []
x_test = []
y_test = []

In [20]:
for i in train_data:
    x_train.append(i[0])
    y_train.append(i[1])

for j in test_data:
    x_test.append(j[0])
    y_test.append(j[1])

In [21]:
from sklearn import linear_model
from sklearn.metrics import mean_squared_error

LR = linear_model.LinearRegression()
LR.fit(x_train, y_train)
y_pred = LR.predict(x_test)
print(mean_squared_error(y_pred, y_test))

0.14577855059070804


In [25]:
x_train[0].shape

(162,)

In [26]:
from sklearn.linear_model import Lasso,ElasticNet,Ridge
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split,KFold,cross_val_score
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import lightgbm as lgb

In [28]:
n_folds=5
print('-----------------------------------','start','----------------------------------------')
lasso=make_pipeline(RobustScaler(),Lasso(alpha=0.0001,random_state=1))
kf=KFold(n_folds,shuffle=True,random_state=42).get_n_splits(x_train)
rmse=np.sqrt(-cross_val_score(lasso,x_train,y_train,scoring='neg_mean_squared_error',cv=kf))
score=rmse
print("\nLasso score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

----------------------------------- start ----------------------------------------


D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increa


Lasso score: 0.3594 (0.0152)



D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


In [42]:
import torch 
import torchvision
from torch import optim,nn

In [43]:
def train_k_fold_data(net,num_epochs,lr,train_features,train_labels,test_features,test_labels,batch_size,montum,wd):
#     net=nn.Linear(train_features.shape[1],1)
#     net=nn.Sequential(nn.Linear(train_features.shape[1],1))
    loss=nn.MSELoss()
#     optimizer=optim.SGD(net.parameters(),lr=lr,momentum=montum,weight_decay=wd)
    optimizer=optim.Adam(net.parameters(),lr=lr,weight_decay=wd)
    dataset=torch.utils.data.TensorDataset(train_features,train_labels)
    data_iter=torch.utils.data.DataLoader(dataset,batch_size,shuffle=True)
    train_l,test_l=[],[]
    
    min_test_loss=1000
    early_stop_cnt=0
    train_loss,test_loss=0,0
    
    for e in range(num_epochs):
        net.train()
        for x,y in data_iter:
            l=loss(net(x),y)
            optimizer.zero_grad()
            l.backward()
            optimizer.step()
#         if (e+1) %1000==0 and test_features is not  None:
        net.eval()
        with torch.no_grad():
            test_loss=loss(net(test_features),test_labels).item()
            if test_loss<min_test_loss:
                min_test_loss=test_loss
#                 test_l.append(test_loss)
                train_loss=loss(net(train_features),train_labels).item()
                test_l.append(test_loss)
                train_l.append(train_loss)
                print('epoch = %d train_loss : %f , test loss : %f' % (e+1,train_loss,test_loss))
                early_stop_cnt=0
            else:
                early_stop_cnt+=1
        if early_stop_cnt > 500:
            
            break
                
#             net.eval()
#             with torch.no_grad():
#                 train_l.append(loss(net(train_features),train_labels).item())
#                 test_l.append(loss(net(test_features),test_labels).item())
# #                 print('epoch ',(e+1),'train loss : ',train_l[-1],'test loss : ',test_l[-1])

    return train_l,test_l

In [44]:
def get_kfold_data(k,j,x,y,random_state=13):
    assert k>=1, 'k must >=1'
    fold_size = x.shape[0] // k
    x_train,y_train=None,None
    row_list=list(range(x.shape[0]))
    np.random.seed(random_state)
    np.random.shuffle(row_list)
    for i in range(k):
        idx=slice(fold_size*i,fold_size*(i+1))
        x_part,y_part=x[row_list[idx],:],y[row_list[idx],:]
        if i==j:
            x_val,y_val=x_part,y_part
        elif x_train is None:
            x_train,y_train=x_part,y_part
        else:
            x_train=torch.cat((x_train,x_part))
            y_train=torch.cat((y_train,y_part))
    return x_train,y_train,x_val,y_val

In [45]:
def train_kfold(net1,num_epochs,lr,k,x_data,y_data,batch_size,montum,wd):
    train_sum_l,test_sum_l=[],[]
    train_l,test_l=[],[]
#     net=nn.Linear(x_data.shape[1])
    for j in range(k):
        net1=get_net()
#         for p in net1.parameters():
#             torch.nn.init.normal_(p)
        net=net1
        data=get_kfold_data(k,j,x_data,y_data)
        train_l,test_l=train_k_fold_data(net,num_epochs,lr,*data,batch_size,montum,wd)
        train_sum_l.append(train_l[-1])
        test_sum_l.append(test_l[-1])
        print('--------------------------------------------------------------------------')
        print('fold %d,train loss mean : %f,test loss : %f' % (j+1,train_l[-1],test_l[-1]))
        print('-------------------------------------------------------------------------')
    print('%d fold ,total train loss mean : %f,total test loss mean : %f ' % (k,np.mean(train_sum_l),np.mean(test_sum_l)))
    print('-----------------------------------------------------------------------------')

In [46]:
len(y_train)

5251

In [47]:
x1=np.empty((5251,162))
y1=np.empty((5251,1))

In [48]:
for i in range(len(y_train)):
    x1[i,:]=x_train[i].reshape(1,-1)
    y1[i,:]=y_train[i].reshape(1,-1)
    

In [49]:
x11=torch.Tensor(x1)
y11=torch.Tensor(y1)

In [50]:
def get_net():
    return nn.Sequential(nn.Linear(x11.shape[1],128),nn.Linear(128,1))
net1=get_net()
train_kfold(net1,1000,0.001,5,x11,y11,256,0,0)

epoch = 1 train_loss : 0.330142 , test loss : 0.334362
epoch = 2 train_loss : 0.252079 , test loss : 0.275748
epoch = 3 train_loss : 0.212178 , test loss : 0.224603
epoch = 4 train_loss : 0.192076 , test loss : 0.204036
epoch = 5 train_loss : 0.175381 , test loss : 0.186812
epoch = 6 train_loss : 0.165759 , test loss : 0.174996
epoch = 7 train_loss : 0.153927 , test loss : 0.164367
epoch = 9 train_loss : 0.142621 , test loss : 0.150846
epoch = 10 train_loss : 0.141071 , test loss : 0.148699
epoch = 11 train_loss : 0.137886 , test loss : 0.145355
epoch = 12 train_loss : 0.140354 , test loss : 0.140552
epoch = 14 train_loss : 0.136109 , test loss : 0.139868
epoch = 16 train_loss : 0.130760 , test loss : 0.137073
epoch = 17 train_loss : 0.130484 , test loss : 0.135370
epoch = 19 train_loss : 0.129300 , test loss : 0.133484
epoch = 20 train_loss : 0.125682 , test loss : 0.132210
epoch = 21 train_loss : 0.126706 , test loss : 0.131220
epoch = 33 train_loss : 0.123907 , test loss : 0.127308
